In [191]:
import pandas as pd
import plotly.express as px
import numpy as np

In [192]:
data = pd.read_parquet(
    "../../data/2_clean/literacy_rates.parquet",
)

In [198]:
data.query("Sexe != 'Total'").info()

<class 'pandas.core.frame.DataFrame'>
Index: 323 entries, 0 to 677
Data columns (total 24 columns):
 #   Column                                                                                                        Non-Null Count  Dtype   
---  ------                                                                                                        --------------  -----   
 0   Pays et territoires insulaires du Pacifique                                                                   323 non-null    category
 1   Sexe                                                                                                          323 non-null    category
 2   Âge                                                                                                           323 non-null    category
 3   Urbanisation                                                                                                  323 non-null    category
 4   Invalidité                                             

In [117]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 708 entries, 0 to 707
Data columns (total 24 columns):
 #   Column                                                                                                        Non-Null Count  Dtype   
---  ------                                                                                                        --------------  -----   
 0   Pays et territoires insulaires du Pacifique                                                                   708 non-null    category
 1   Sexe                                                                                                          708 non-null    category
 2   Âge                                                                                                           708 non-null    category
 3   Urbanisation                                                                                                  708 non-null    category
 4   Invalidité                                        

In [190]:
data.set_index(["TIME_PERIOD", "Pays et territoires insulaires du Pacifique", "Sexe"])[
    "Taux d'alphabétisation des 15-24 ans (%)"
].dropna().reset_index()

,TIME_PERIOD,Pays et territoires insulaires du Pacifique,Sexe,Taux d'alphabétisation des 15-24 ans (%)
0,1986,Fidji,Total,97.5
1,1996,Fidji,Total,99.3
2,2008,Fidji,Total,99.5
3,2014,Guam,Total,0.0
4,2020,Kiribati,Femme,97.4
...,...,...,...,...
219,2021,Îles Marshall (pays),Total,65.2
220,2021,Îles Marshall (pays),Total,88.5
221,2021,Îles Marshall (pays),Total,93.9
222,2021,Îles Marshall (pays),Total,63.9


## By year

---


In [119]:
alphabetisation = (
    data.groupby(["TIME_PERIOD", "Sexe"])["Taux d'alphabétisation des 15-24 ans (%)"]
    .mean()
    .dropna()
    .reset_index()
)
alphabetisation = alphabetisation.pivot_table(
    index="TIME_PERIOD",
    columns="Sexe",
    values="Taux d'alphabétisation des 15-24 ans (%)",
)
alphabetisation = alphabetisation.replace(0, np.nan).ffill().bfill()
alphabetisation = alphabetisation.stack()
alphabetisation.name = "Taux d'alphabétisation des 15-24 ans (%)"
alphabetisation = alphabetisation.reset_index()
alphabetisation

/var/folders/36/grrgsqjd14j4tf6cf5ty4ykh0000gn/T/ipykernel_15683/3719543329.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/36/grrgsqjd14j4tf6cf5ty4ykh0000gn/T/ipykernel_15683/3719543329.py:7: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



,TIME_PERIOD,Sexe,Taux d'alphabétisation des 15-24 ans (%)
0,1986,Femme,88.766667
1,1986,Homme,88.633333
2,1986,Total,97.500000
3,1990,Femme,88.766667
4,1990,Homme,88.633333
...,...,...,...
67,2020,Homme,87.304167
68,2020,Total,84.915385
69,2021,Femme,85.761905
70,2021,Homme,81.604762


In [30]:
px.line(
    alphabetisation.query("TIME_PERIOD >= 2010"),
    x="TIME_PERIOD",
    y="Taux d'alphabétisation des 15-24 ans (%)",
    color="Sexe",
    # fill area bellows the line
    line_shape="spline",
    title="Taux d'alphabétisation des 15-24 ans (%) par sexe",
    labels={
        "TIME_PERIOD": "Année",
        "Taux d'alphabétisation des 15-24 ans (%)": "Taux d'alphabétisation des 15-24 ans (%)",
    },
)

In [143]:
alphabetisation_diff = alphabetisation.query("Sexe == 'Homme'").drop(
    columns="Sexe"
).set_index("TIME_PERIOD") - alphabetisation.query("Sexe == 'Femme'").drop(
    columns="Sexe"
).set_index("TIME_PERIOD")
alphabetisation_diff = alphabetisation_diff.query("TIME_PERIOD >= 2010")
alphabetisation_diff["Sens"] = (
    alphabetisation_diff["Taux d'alphabétisation des 15-24 ans (%)"] > 0
).replace({True: "Plus d'hommes", False: "Plus de femmes"})
alphabetisation_diff["Taux d'alphabétisation des 15-24 ans (%)"] = alphabetisation_diff[
    "Taux d'alphabétisation des 15-24 ans (%)"
].abs()
alphabetisation_diff

,Taux d'alphabétisation des 15-24 ans (%),Sens
TIME_PERIOD,,
2010,0.133333,Plus de femmes
2011,1.866667,Plus de femmes
2012,1.866667,Plus de femmes
2013,2.760000,Plus de femmes
2014,2.760000,Plus de femmes
2015,11.836667,Plus d'hommes
2018,0.240000,Plus de femmes
2020,0.731090,Plus d'hommes
2021,4.157143,Plus de femmes


In [144]:
px.bar(
    alphabetisation_diff,
    y="Taux d'alphabétisation des 15-24 ans (%)",
    color="Sens",
    title="Différence de taux d'alphabétisation des 15-24 ans (%) entre les hommes et les femmes",
    labels={
        "Taux d'alphabétisation des 15-24 ans (%)": "Différence (%)",
    },
)

## By country

---


In [43]:
alphabetisation_by_country = (
    data.groupby(["Pays et territoires insulaires du Pacifique", "Sexe"])[
        "Taux d'alphabétisation des 15-24 ans (%)"
    ]
    .mean()
    .dropna()
    .reset_index()
)
alphabetisation_by_country = alphabetisation_by_country.query("Sexe != 'Total'")
alphabetisation_by_country

/var/folders/36/grrgsqjd14j4tf6cf5ty4ykh0000gn/T/ipykernel_15683/2939287501.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,Pays et territoires insulaires du Pacifique,Sexe,Taux d'alphabétisation des 15-24 ans (%)
2,Kiribati,Femme,86.055556
3,Kiribati,Homme,82.955556
5,Nauru,Femme,85.560000
6,Nauru,Homme,84.520000
10,Palaos,Femme,87.540909
11,Palaos,Homme,98.830000
16,Samoa,Femme,98.700000
17,Samoa,Homme,97.000000
20,Tonga,Femme,88.211111
21,Tonga,Homme,84.300000


In [184]:
px.sunburst(
    alphabetisation_by_country,
    path=["Pays et territoires insulaires du Pacifique", "Sexe"],
    values="Taux d'alphabétisation des 15-24 ans (%)",
    title="Taux d'alphabétisation des 15-24 ans (%) par pays et sexe",
)

/Users/ash/Documents/Workspaces/Python/Data-viz-challenge-2024/.venv/lib/python3.12/site-packages/plotly/express/_core.py:1706: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/Users/ash/Documents/Workspaces/Python/Data-viz-challenge-2024/.venv/lib/python3.12/site-packages/plotly/express/_core.py:1706: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [109]:
alphabetisation_by_country_diff = alphabetisation_by_country.set_index(
    ["Pays et territoires insulaires du Pacifique", "Sexe"]
)
alphabetisation_by_country_diff = alphabetisation_by_country_diff.query(
    "Sexe == 'Homme'"
)["Taux d'alphabétisation des 15-24 ans (%)"].droplevel(
    1
) - alphabetisation_by_country_diff.query("Sexe == 'Femme'")[
    "Taux d'alphabétisation des 15-24 ans (%)"
].droplevel(1)
alphabetisation_by_country_diff = alphabetisation_by_country_diff.reset_index()
alphabetisation_by_country_diff["Sens"] = (
    alphabetisation_by_country_diff["Taux d'alphabétisation des 15-24 ans (%)"] > 0
).replace({True: "plus d'hommes", False: "plus de femmes"})

alphabetisation_by_country_diff["abs_value"] = alphabetisation_by_country_diff[
    "Taux d'alphabétisation des 15-24 ans (%)"
].abs()
alphabetisation_by_country_diff = alphabetisation_by_country_diff.sort_values(
    "abs_value", ascending=False
)
alphabetisation_by_country_diff

,Pays et territoires insulaires du Pacifique,Taux d'alphabétisation des 15-24 ans (%),Sens,abs_value
2,Palaos,11.289091,plus d'hommes,11.289091
8,Îles Marshall (pays),-5.644444,plus de femmes,5.644444
4,Tonga,-3.911111,plus de femmes,3.911111
0,Kiribati,-3.100000,plus de femmes,3.100000
5,Vanuatu,-2.111111,plus de femmes,2.111111
3,Samoa,-1.700000,plus de femmes,1.700000
6,Wallis-et-Futuna,-1.500000,plus de femmes,1.500000
1,Nauru,-1.040000,plus de femmes,1.040000
7,États fédérés de Micronésie (pays),-0.133333,plus de femmes,0.133333


In [112]:
px.bar(
    alphabetisation_by_country_diff,
    x="Pays et territoires insulaires du Pacifique",
    y="abs_value",
    color="Sens",
    title="Différence de taux d'alphabétisation des 15-24 ans (%) entre les hommes et les femmes",
    labels={
        "abs_value": "Différence (%)",
        "Pays et territoires insulaires du Pacifique": "Pays et territoires insulaires du Pacifique",
    },
)